In [19]:
from octopus import OctopusClient

import networkx as nx

In [20]:
def build_nx_graph():
    """
    Builds and returns networkx graph for Movies.

    Parameters : 
    None

    Returns : 
    networkx graph object
        A networkx graph object of Movies
    """
    import csv 
    
    def remove_escape_quotes(s):
        return s.replace('\\"', '"')

    movies_csv_path = "../utils/movies.csv"
    G = nx.Graph()

    with open(movies_csv_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for row in reader:
            # Check if it's a node or relationship
            if row[0]:  # Node
                #print(row[0])
                node_type = row[1].split(":")[1]  # Extract node type from labels
                if node_type == "Person":
                    G.add_node(row[0], label="Person", born=row[2], name=remove_escape_quotes(row[3]))
                elif node_type == "Movie":
                    G.add_node(row[0], label="Movie", released=row[4], tagline=remove_escape_quotes(row[5]), title=remove_escape_quotes(row[6]))
            else:  # Relationship
                rel_type = row[9]
                if rel_type == "ACTED_IN":
                    G.add_edge(row[7], row[8], label="ACTED_IN", rating=remove_escape_quotes(row[10]), roles=remove_escape_quotes(row[11]))
                elif rel_type == "WROTE":
                    G.add_edge(row[7], row[8], label="WROTE")
                elif rel_type == "PRODUCED":
                    G.add_edge(row[7], row[8], label="PRODUCED")
                elif rel_type == "DIRECTED":
                    G.add_edge(row[7], row[8], label="DIRECTED")
                elif rel_type == "REVIEWED":
                    G.add_edge(row[7], row[8], label="REVIEWED", rating=remove_escape_quotes(row[10]), summary=remove_escape_quotes(row[11]))

    return G

In [21]:
def task_1_nx(nx_graph):
    """
    Using networkx library, find the actor named "Tom Hanks".

    Parameters:
    nx_graph : networkx object
       Movies graph
    
    Returns :
    None
    """
    for node, attrs in nx_graph.nodes(data=True):
        if attrs.get("name") == "Tom Hanks":
            print(node, attrs)
            break

In [22]:
def task_1_octopus(oc):
    """
    Using Octopus, find the actor named "Tom Hanks".

    Parameters:
    oc : Octopus Client
        Octopus Client
    
    Returns :
    None
    """
    query = "MATCH (n:Person) where n.name = \"Tom Hanks\" RETURN n"
    execution = oc.execute(query, 1)
    execution.poll()
    print(execution.output.fetch_all())

In [23]:
def task_2_nx(nx_graph):
    """
    Using networkx library, find all movies released in the 90s and print their titles.

    Parameters:
    nx_graph : networkx object
       Movies graph
    
    Returns :
    None
    """
    movies_90s = [node for node, attrs in nx_graph.nodes(data=True) 
              if attrs.get("label") == "Movie" and attrs.get("released") and attrs.get("released").startswith("199")]

    for movie in movies_90s:
        for node, attrs in nx_graph.nodes(data=True):
            if movie == node:
                print(attrs.get("title"))
    

In [24]:
def task_2_octopus(oc):
    """
    Using Octopus, find all movies released in the 90s and print their titles.

    Parameters:
    oc : Octopus Client
        Octopus Client
        
    Returns :
    None
    """
    query = "MATCH (n:Movie) where n.released < 2000 and n.released >= 1990 RETURN n.title"
    execution = oc.execute(query, 1)
    execution.poll()
    print(execution.output.fetch_all())  

In [25]:
def task_3_nx(nx_graph):
    """
    Using networkx library, find how people are related to the movie "Cloud Atlas".

    Parameters:
    nx_graph : networkx object
       Movies graph
    
    Returns :
    None
    """
    cloud_atlas = None    
    for node, attrs in nx_graph.nodes(data=True):
        if attrs.get("title") == "Cloud Atlas":
           cloud_atlas = node
           break

    # networkx sort node depending on order, even if you loaded the node as end, it will be stored as start if its id is lower than the other node's id
    for start, end, attrs1 in nx_graph.edges(data=True):
        key = None
        if start == cloud_atlas:
           key = end
        elif end == cloud_atlas:
           key = start
        if key:
           roles = ""
           if "roles" in attrs1:
               roles = attrs1["roles"] 
           for node, attrs2 in nx_graph.nodes(data=True):
               if node == key:
                  print(attrs2["name"], attrs1["label"], roles)


In [26]:
def task_3_octopus(nx_graph):
    """
    Using Octopus, find how people are related to the movie "Cloud Atlas".

    Parameters:
     oc : Octopus Client
        Octopus Client
    
    Returns :
    None
    """
    query = "MATCH (people:Person)-[relatedTo]-(:Movie {title: \"Cloud Atlas\"}) RETURN people.name, Type(relatedTo), relatedTo.roles"
    execution = oc.execute(query, 1)
    execution.poll()
    print(execution.output.fetch_all())  

In [27]:
if __name__ == "__main__":
    # Build networkx graph.
    #nx_graph = build_nx_graph()

    # Instantiate Octopus Client
    #oc = OctopusClient("localhost", "8000", "neo4j", "password")

    # Task-1
    #print("Task-1: Find the actor named \"Tom Hanks\"")
    #print("networkx:")
    #task_1_nx(nx_graph)
    #print("Octopus:")
    #task_1_octopus(oc)

    # Task-2
    #print("Task-2: Find movies released in the 90s and return their titles")
    #print("networkx:")
    #task_2_nx(nx_graph)
    #print("Octopus:")
    #task_2_octopus(oc)

    # Task-3
    #print("Task-3: How people are related to \"Cloud Atlas\"?")
    #print("networkx:")
    #task_3_nx(nx_graph)
    #print("Octopus:")
    #task_3_octopus(oc)    
    